# Aula 6 - Regularização

Neste ponto, é muito importante que falemos sobre **regularização**.

O objetivo da regularização é **diminuir a complexidade** de modelos, de modo a evitar que particularidades da base de treino (ruídos) sejam aprendidos (ou seja, evitar overfitting!)

Uma outra forma de enxergar regularização: **diminuição do espaço de hipóteses!**

<img src=https://curso-r.github.io/main-intro-ml/slides/static/img/erro_treino_erro_teste.png width=500>

Regularização: problema de otimização VINCULADO!! ou seja, com restrições.

problema de otimização: otimização da função de custo, que é o objetivo da aprendizagem, pra determinar o $\hat{\vec{b}}$

restrições: é o que determina se temos L1 (lasso) ou L2 (ridge)

### Regressão linear (sem regularização)

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/5408b0a7-85f3-4824-ad68-44867121ecb9.png width=800>

### L1 (Lasso)

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/acabe9da-07ba-4337-b467-dd2701a40cc8.png width=900>

### L2 (Ridge)

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/46eda310-fb2f-498b-b455-593183de1dd7.png width=900>

Para saber como relacionar $t$ com $\lambda$, veja [este post](https://stats.stackexchange.com/questions/259177/expressing-the-lasso-regression-constraint-via-the-penalty-parameter) ou então [este](https://stats.stackexchange.com/questions/90648/kkt-versus-unconstrained-formulation-of-lasso-regression) -- discussões bem matemáticas!

Observações importantes:

- $\lambda$ é um parâmetro que controla a "força" da regularização<br><br>
- **L1 pode zerar coeficientes** - faz feature selection<br><br>
- **L2 apenas diminui o tamanho de coeficientes** - não faz feature selection<br><br>

No sklearn, é possível fazer um modelo de regressão linear regularizado facilmente com as classes respectivas:

- [Regularização L2/Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge)

- [Regularização L1/Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso)

Há, no sklearn, também uma implementação para um tipo de regularização conhecida como **Elastic Net**:

A classe se chama [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet)

Vamos utilizar regularização no dataset das casas, juntamente com as features polinomiais:

> **IMPORTANTE**: como os métodos de regularização são baseados na norma do vetor de parâmetros, é muito importante que as features sejam escaladas para que os métodos funcionem bem!

Isso porque a escala das features irá influenciar a regularização aplicada ao parâmetro respectivo!

Para eliminar este efeito, escalar os dados é muito importante!

Vamos visualizar concretamente como a regularização de fato simplifica a hipótese! Pra isso, considere os pontos a seguir:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
def poly_regularized_reg(X, y, degree, 
                         type_regularization=None, alpha=1, l1_ratio=0.5, 
                         iter_max=1000):
    '''
    - type_regularization (str): opções de regularização: ["l1", "l2", "en", None]
    '''

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # ======================================
 
    pf = PolynomialFeatures(degree=degree, include_bias=False).fit(X_train)

    # redefinindo as features de treino e de teste
    X_train = pf.transform(X_train)
    X_test = pf.transform(X_test)
    
    print(f"Número original de features: {pf.n_features_in_}")
    print(f"Número de features no espaço transformado: {pf.n_output_features_}")

    # ======================================
    # normalização dos dados - MUITO importante quando há regularização!!
    # e é o passo imediatamente antes de treinar os modelos
    
    mms = MinMaxScaler().fit(X_train)
    
    X_train = mms.transform(X_train)
    X_test = mms.transform(X_test)
    
    # ======================================

    if type_regularization == "l1":
        
        model = Lasso(alpha=alpha, max_iter=iter_max).fit(X_train, y_train)
        
    elif type_regularization == "l2":
        
        model = Ridge(alpha=alpha, max_iter=iter_max).fit(X_train, y_train)
        
    elif type_regularization == "en":
        
        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=iter_max).fit(X_train, y_train)
        
    elif type_regularization == None:
    
        model = LinearRegression().fit(X_train, y_train)
        
    else:
        
        list_opcoes = ["l1", "l2", "en", None]
        raise ValueError(f"Opção de regularização indisponível!\nOpções aceitas: {list_opcoes}")


    # ======================================

    # predições de treino
    y_pred_train = model.predict(X_train)

    print("\nMétricas de treino:\n")
    print(f"R^2: {r2_score(y_train, y_pred_train):.2f}")
    print(f"MAE: {mean_absolute_error(y_train, y_pred_train):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train)):.2f}")

    # predições de teste
    y_pred_test = model.predict(X_test)

    print("\nMétricas de teste:\n")
    print(f"R^2: {r2_score(y_test, y_pred_test):.2f}")
    print(f"MAE: {mean_absolute_error(y_test, y_pred_test):.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}") 

In [ ]:
df = pd.read_csv("house_prices.csv")
df = df.select_dtypes(include=np.number).dropna()

X = df.drop(columns=["SalePrice", "Id"])
y = df["SalePrice"]

In [ ]:
# regressão linear, sem regularização
# exatamente o que fizemos na primeira aula!
poly_regularized_reg(X, y, degree=1, type_regularization=None)

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization=None)

### L1

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="l1", alpha=100, iter_max=2000)

In [ ]:
poly_regularized_reg(X, y, degree=3, type_regularization="l1", alpha=100, iter_max=2000)

### L2

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="l2", alpha=200, iter_max=2000)

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="l2", alpha=50, iter_max=2000)

In [ ]:
poly_regularized_reg(X, y, degree=3, type_regularization="l2", alpha=50, iter_max=2000)

### Elastic Net

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="en", 
                     alpha=1, l1_ratio=0.5,
                     iter_max=2000)

In [ ]:
poly_regularized_reg(X, y, degree=2, type_regularization="en", 
                     alpha=10, l1_ratio=0.8,
                     iter_max=2000)

In [ ]:
poly_regularized_reg(X, y, degree=3, type_regularization="en", 
                     alpha=100, l1_ratio=0.8,
                     iter_max=2000)